# Model Serialization and Chain Storage in LangChain

이 노트북에서는 LangChain의 모델과 체인을 직렬화하고 저장하는 방법을 학습합니다.

## 직렬화(Serialization) 개념

### 직렬화(Serialization)란?

1. **정의:**
   - 모델을 저장 가능한 형식으로 변환하는 과정

2. **목적:**
   - 모델 재사용 (재훈련 없이)
   - 모델 배포 및 공유 용이
   - 계산 리소스 절약

3. **장점:**
   - 빠른 모델 로딩
   - 버전 관리 가능
   - 다양한 환경에서 사용 가능

모델 직렬화는 AI 개발 및 배포 과정에서 중요한 단계로, 효율적인 모델 관리와 재사용을 가능하게 합니다.

`is_lc_serializable` 클래스 메서드로 실행하여 LangChain 클래스가 직렬화 가능한지 확인할 수 있습니다.

## 환경 설정

In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 직렬화 가능 여부 확인

LangChain의 컴포넌트들이 직렬화 가능한지 확인하는 방법을 알아봅니다.

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿을 사용하여 질문을 생성합니다.
prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

In [3]:
# 클래스(class) 에 대하여 직렬화 가능 여부를 확인합니다.
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {llm.is_lc_serializable()}")

ChatOpenAI: True


In [5]:
# 체인을 생성합니다.
chain = prompt | llm

# 직렬화가 가능한지 체크합니다.
chain.is_lc_serializable()

True

## 체인(Chain) 직렬화 - dumps와 dumpd

### 개요

체인 직렬화는 직렬화 가능한 모든 객체를 딕셔너리 또는 JSON 문자열로 변환하는 과정을 의미합니다.

### 직렬화 방법

객체의 속성 및 데이터를 키-값 쌍으로 저장하여 딕셔너리 형태로 변환합니다.

이러한 직렬화 방식은 객체를 쉽게 저장하고 전송할 수 있게 하며, 다양한 환경에서 객체를 재구성할 수 있도록 합니다.

**참고**
- `dumps`: 객체를 JSON 문자열로 직렬화
- `dumpd`: 객체를 딕셔너리로 직렬화

In [6]:
from langchain_core.load import dumpd, dumps

# 체인을 딕셔너리로 직렬화
dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate',
   'graph': {'nodes': [{'id': 0, 'type': 'schema', 'data': 'PromptInput'},
     {'id': 1,
      'type': 'runnable',
      'data': {'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
       'name': 'PromptTemplate'}},
     {'id': 2, 'type': 'schema', 'data': 'PromptTemplateOutput'}],
    'edges': [{'source': 0, 'target': 1}, {'source': 1, 'target': 2}]}},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'openai_p

In [7]:
# 직렬화된 체인의 타입을 확인합니다.
type(dumpd_chain)

dict

## Pickle 파일로 저장하기

### Pickle 파일 개요

Pickle 파일은 Python 객체를 바이너리 형태로 직렬화하는 포맷입니다.

### 특징

1. **형식:**
   - Python 객체를 바이너리 형태로 직렬화하는 포맷

2. **특징:**
   - Python 전용 (다른 언어와 호환 불가)
   - 대부분의 Python 데이터 타입 지원 (리스트, 딕셔너리, 클래스 등)
   - 객체의 상태와 구조를 그대로 보존

3. **장점:**
   - 효율적인 저장 및 전송
   - 복잡한 객체 구조 유지
   - 빠른 직렬화/역직렬화 속도

4. **단점:**
   - 보안 위험 (신뢰할 수 없는 데이터 역직렬화 시 주의 필요)
   - 사람이 읽을 수 없는 바이너리 형식

### 주요 용도

1. 객체 캐싱
2. 머신러닝 모델 저장
3. 프로그램 상태 저장 및 복원

### 사용법

- `pickle.dump()`: 객체를 파일에 저장
- `pickle.load()`: 파일에서 객체 로드

In [8]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)

## JSON 파일로 저장하기

In [9]:
import json

# JSON 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.json", "w") as fp:
    json.dump(dumpd_chain, fp)

## 저장된 체인 로드 및 실행

### Pickle 파일에서 로드하기

In [10]:
import pickle

# pickle 파일을 로드합니다.
with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)

In [11]:
from langchain_core.load import load

# 체인을 로드합니다.
chain_from_file = load(loaded_chain)

# 체인을 실행합니다.
print(chain_from_file.invoke({"fruit": "사과"}))

C:\Users\cava2\AppData\Local\Temp\ipykernel_18408\741098595.py:4: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  chain_from_file = load(loaded_chain)


content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 24, 'total_tokens': 76, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-cfa8b4d2-0d70-449b-8d54-86c61184fa6e-0' usage_metadata={'input_tokens': 24, 'output_tokens': 52, 'total_tokens': 76}


### API 키와 함께 체인 로드하기

In [12]:
from langchain_core.load import load, loads

# secrets_map을 사용하여 API 키를 명시적으로 전달
load_chain = load(
    loaded_chain, secrets_map={"OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}
)

# 불러온 체인이 정상 동작하는지 확인합니다.
load_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 24, 'total_tokens': 76, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-859b6a21-2621-45fd-b423-e968bbb57e1e-0', usage_metadata={'input_tokens': 24, 'output_tokens': 52, 'total_tokens': 76})

### JSON 파일에서 로드하기

In [13]:
# JSON 파일에서 체인을 로드합니다.
with open("fruit_chain.json", "r") as fp:
    loaded_from_json_chain = json.load(fp)
    loads_chain = load(loaded_from_json_chain)

In [14]:
# 불러온 체인이 정상 동작하는지 확인합니다.
loads_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 24, 'total_tokens': 76, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-82aadafe-ebc6-4d99-81d0-26e69b0c37c6-0', usage_metadata={'input_tokens': 24, 'output_tokens': 52, 'total_tokens': 76})

## 요약

이 노트북에서는 다음 내용을 학습했습니다:

1. **직렬화 개념과 중요성**
   - 모델을 저장 가능한 형식으로 변환
   - 재사용성과 배포 용이성 향상

2. **LangChain 컴포넌트의 직렬화 가능 여부 확인**
   - `is_lc_serializable()` 메서드 사용

3. **체인 직렬화 방법**
   - `dumpd()`: 딕셔너리로 직렬화
   - `dumps()`: JSON 문자열로 직렬화

4. **저장 방법**
   - Pickle 파일: 바이너리 형식, Python 전용
   - JSON 파일: 텍스트 형식, 다른 언어와 호환 가능

5. **로드 및 실행**
   - `load()` 함수를 사용한 체인 복원
   - API 키 관리를 위한 `secrets_map` 사용

이러한 직렬화 기능을 통해 훈련된 모델과 체인을 효율적으로 저장하고 재사용할 수 있습니다.